In [ ]:
'''
CREATE TABLE ko_gamelog (
    gameid INT,
    atBatIndex INT,
    type VARCHAR(20),
    awayScore INT,
    homeScore INT,
    batterid INT,
    batterName VARCHAR(50),
    pitcherid INT,
    pitcherName VARCHAR(50),
    inning INT,
    halfInning VARCHAR(10),
    runnerIndex INT,
    ball INT,
    strike INT, 
    `out` INT,
    RBI INT,
    `event` VARCHAR(500),
    situation VARCHAR(5)
)
'''

Get all games for 2024 season

In [4]:
import statsapi
from game import game
from team import team
sched = statsapi.schedule(start_date='01/01/2024',end_date='12/30/2024')
# team id key dict
t_info = {}

t = team()
t.getAll()
for tt in t.data:
    t_info[tt["teamid"]] = tt['teamKey']
    
game_list = []
for i in sched:
    if i['status'] == 'Final' and i['game_type'] == "R":
        g_info = {}
        g_get = statsapi.get('game', {'gamePk': i['game_id']})
        g_info['awayWins'] = g_get["gameData"]['teams']['away']['record']['leagueRecord']['wins']
        g_info['awayLosses'] = g_get["gameData"]['teams']['away']['record']['leagueRecord']['losses']
        g_info['homeWins'] = g_get["gameData"]['teams']['home']['record']['leagueRecord']['wins']
        g_info['homeLosses'] = g_get["gameData"]['teams']['home']['record']['leagueRecord']['losses']
        # print(g_info['awayWins'],g_info['awayLosses'])
        g_info['gameid'] = i['game_id']
        g_info['date'] = i['game_date']
        g_info['awayTeam'] = i['away_name']
        g_info['homeTeam'] = i['home_name']
        g_info['awayKey'] = t_info[i['away_id']]
        g_info['homeKey'] = t_info[i['home_id']]
        g_info['awayid'] = i['away_id']
        g_info['homeid'] = i['home_id']
        g_info['homeScore'] = i['home_score']
        g_info['awayScore'] = i['away_score']
        game_list.append(g_info)
        # print(i)

g = game()
g.truncate()
count = 0
sql = f"INSERT INTO `{g.tn}` ("
for field in g.fields:
    sql += f"`{field}`,"
    count +=1
sql = sql[0:-1] + ') VALUES ('
tokens = ("%s," * count)[0:-1]
sql += tokens + ');'
print(sql)
n = 0
vals = []
for g_data in game_list:
    val = []
    for field in g.fields:
        val.append(g_data[field])
    vals.append(val)
    if n % 100 == 0:
        g.cur.executemany(sql,vals)
        vals = []
if len(vals) > 0:
    g.cur.executemany(sql,vals)


INSERT INTO `ko_game` (`gameid`,`date`,`homeKey`,`homeTeam`,`homeid`,`homeWins`,`homeLosses`,`awayKey`,`awayTeam`,`awayid`,`awayWins`,`awayLosses`,`homeScore`,`awayScore`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);


Get the gamelogs for every game

In [58]:
import statsapi
from gamelog import gamelog
g = gamelog()
g.truncate()
count = 0
sql = f"INSERT INTO `{g.tn}` ("
for field in g.fields:
    sql += f"`{field}`,"
    count +=1
sql = sql[0:-1] + ') VALUES ('
tokens = ("%s," * count)[0:-1]
sql += tokens + ');'

for game in game_list:
    g_get = statsapi.get('game', {'gamePk': game['gameid']})
    game_log = []
    score_dif = 0
    base = 0
    out = 0
    halfinning = ""
    for t in g_get['liveData']['plays']['allPlays']:
        # record inning change
        if halfinning != t['about']['halfInning']:
            halfinning = t['about']['halfInning']
            # print(t['about']['halfInning'])
            # print(t['about']['inning'])
            res = {}
            res['gameid'] = game['gameid']
            res['type'] = 'inning change'
            res['atBatIndex'] = 999
            res['batterid'] = 0
            res['batterName'] = ''
            res['pitcherid'] = 0
            res['pitcherName'] = ''
            res['inning'] = t['about']['inning']
            res['halfInning'] = t['about']['halfInning']
            res['awayScore'] = 0
            res['homeScore'] = 0
            res['ball'] = 0
            res['strike'] = 0
            res['out'] = 0
            res['RBI'] = 0
            res['runnerIndex'] = 0
            res['event'] = 'inning change'
            res['situation'] = ""
            res['situation_after'] = ""
            game_log.append(res)


        res = {}
        res['gameid'] = game['gameid']
        res['type'] = 'play'
        res['atBatIndex'] = t['atBatIndex']
        res['batterid'] = t['matchup']['batter']['id']
        res['batterName'] = t['matchup']['batter']['fullName']
        res['pitcherid'] = t['matchup']['pitcher']['id']
        res['pitcherName'] = t['matchup']['pitcher']['fullName']
        res['inning'] = t['about']['inning']
        res['halfInning'] = t['about']['halfInning']
        res['awayScore'] = t['result']['awayScore']
        res['homeScore'] = t['result']['homeScore']
        res['ball'] = t['count']['balls']
        res['strike'] = t['count']['strikes']
        res['out'] = t['count']['outs']
        res['RBI'] = t['result']['rbi']
        res['runnerIndex'] = 0

        n_list = []
        for r in t['runners'][::-1]:
            if r['details']['runner']['id'] not in n_list:
                if r['movement']['end'] == '1B':
                    res['runnerIndex'] += 1
                elif r['movement']['end'] == '2B':
                    res['runnerIndex'] += 2
                elif r['movement']['end'] == '3B':
                    res['runnerIndex'] += 4
                n_list.append(r['details']['runner']['id'])

        n = 0
        for d in t['playEvents']:
            if n in t['actionIndex']:
                if d['details']['event'] != 'Game Advisory':
                    act = {}
                    act = res.copy()
                    act['type'] = 'action'
                    act['event'] = d['details']['description']
                    act['situation'] = ''
                    act['situation_after'] = ''
                    # print(d['details']['event'])
                    game_log.append(act)
            n += 1

        res['event'] = t['result']['event']

        # recording the situation when batter on plate
        if t['about']['isTopInning']:
            res['situation'] = str(score_dif)+str(base)+str(out)
        else:
            res['situation'] = str(-score_dif)+str(base)+str(out)
        # recording the situation after the play
        score_dif = t['result']['awayScore'] - t['result']['homeScore']
        if t['count']['outs'] == 3:
            out = 0
            base = 0
        else:
            out = t['count']['outs']
            base = res['runnerIndex']
        if t['about']['isTopInning']:
            res['situation_after'] = str(score_dif)+str(base)+str(out)
        else:
            res['situation_after'] = str(-score_dif)+str(base)+str(out)
        game_log.append(res)
    vals = []
    for l in game_log:
        val = []
        for field in g.fields:
            val.append(l[field])
        vals.append(val)
    g.cur.executemany(sql,vals)


In [57]:
import statsapi
from gamelog import gamelog
g = gamelog()
g.truncate()